# QUEUES

Queues are the backbone of numerous algorithms found in games, artificial intelligence, satellite navigation, and task scheduling. They’re among the top abstract data types that computer science students learn early in their education. At the same time, software engineers often leverage higher-level message queues to achieve better scalability of a microservice architecture. Plus, using queues in Python is simply fun!

Python provides a few built-in flavors of queues that you’ll see in action in this tutorial. You’re also going to get a quick primer on the theory of queues and their types. Finally, you’ll take a look at some external libraries for connecting to popular message brokers available on major cloud platform providers.

In this tutorial, you’ll learn how to:

    Differentiate between various types of queues
    Implement the queue data type in Python
    Solve practical problems by applying the right queue
    Use Python’s thread-safe, asynchronous, and interprocess queues
    Integrate Python with distributed message queue brokers through libraries


## 004. THIS IS ABOUT...

https://realpython.com/queue-in-python/

In [17]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [18]:
from collections import deque
from heapq import heappop, heappush


### 001.001 Building a Queue Data Type

Because you want your custom FIFO queue to support at least the enqueue and dequeue operations, go ahead and write a bare-bones Queue class that’ll delegate those two operations to `deque``


1. Create a BasicQueue FIFO queue class which
    1. starts with an empty queue
    1. has an `enqueue` method to add items to the queue
    1. has a `deque` method which gets it off it
    1. uncomment the assertions to prove they work
1. Create a slightly more complex Queue class which
    1. can take a list of elements
    1. enqueue and dequeue as before
    1. allows `len(an_instance)` to return the length
    1. allows `for i in an_instance...` to iterate through items
    1. uncomment the assertions to prove they work


In [22]:
1
class BasicQueue:
    def __init__(self):
        self._elements = deque()

    def enqueue(self, element):
        self._elements.append(element)

    def dequeue(self):
        return self._elements.popleft()

1.4
fifo = BasicQueue()
fifo.enqueue(1)
fifo.enqueue(2)
fifo.enqueue(3)

assert fifo.dequeue() == 1
assert fifo.dequeue() == 2
assert fifo.dequeue() == 3

2
class Queue:
    def __init__(self, *elements):
        self._elements = deque(elements)

    def enqueue(self, element):
        self._elements.append(element)

    def dequeue(self):
        return self._elements.popleft()

    def __len__(self):
        return len(self._elements)


    def __iter__(self):
        while len(self._elements):
            yield self.dequeue()

2.5
fifo = Queue(1, 2, 3)
assert len(fifo) == 3
assert [1,2,3] == [el for el in fifo]
assert len(fifo) == 0

fifo.enqueue(1)
fifo.enqueue(2)
fifo.enqueue(3)
assert fifo.dequeue() == 1
assert fifo.dequeue() == 2
assert fifo.dequeue() == 3

# solution


1

1.4

2

2.5

### 001.002 Building a Stack Data Type

Building a stack data type is considerably more straightforward because you’ve already done the bulk of the hard work. Since most of the implementation will remain the same, you can extend your Queue class using inheritance and override the .dequeue() method to remove elements from the top of the stack

(NOTE: this doesn't imply that conceptually Stack extends Queue, it's simply for convenience)

1. Create a Stack class
    1. It inherits from Queue
    1. It overwrites the dequeue method to remove elements from the top
    1. uncomment assertions

In [27]:
1
class Stack(Queue):
    def dequeue(self):
        return self._elements.pop()

lifo = Stack(1, 2, 3)
assert len(lifo) == 3
assert [3,2,1] == [el for el in lifo]
assert len(lifo) == 0

# solution


1

### 001.003 Building a Priority Queue

Fortunately, you can be smart about keeping the elements sorted in a priority queue by using a heap data structure under the hood. It provides the most efficient implementation

Python has the heapq module, which conveniently provides a few functions that can turn a regular list into a heap and manipulate it efficiently.

1. Create a PriorityQueue class
    1. Initialised with 
        1. It stores elements in a simple list
        1. It uses a standard generator from itertools to keep track of next item
        1. ...which needs to be imported
    1. enqueue_with_priority leverages Python’s tuple comparison, which takes into account the tuple’s components, looking from left to right until the outcome is known
        1. Elements contain the priority first...
            1. Q Note that heapq is a min-heap, i.e....
            1. How does that change the way priority is encoded?
            1. We would like to maintain the chronological ordering of queue items if they have the same priority. That's what the counter is all about
            1. Finally the value
        1. Then the element is finally pushed into the heap
    1. Dequeue only fetches the value from the helement, letting heapq do the work
        

In [28]:
from itertools import count

CRITICAL = 3
IMPORTANT = 2
NEUTRAL = 1


1
class PriorityQueue:
    def __init__(self):
        self._elements = []
        self._count = count()

    def enqueue_with_priority(self, priority, value):
        """1.2.1"""
        element = (-priority, next(self._count), value)
        heappush(self._elements, element)

    def dequeue(self):
        return heappop(self._elements)

# 2
messages = PriorityQueue()
messages.enqueue_with_priority(IMPORTANT, "Windshield wipers turned on")
messages.enqueue_with_priority(NEUTRAL, "Radio station tuned in")
messages.enqueue_with_priority(CRITICAL, "Brake pedal depressed")
messages.enqueue_with_priority(IMPORTANT, "Hazard lights turned on")

# assert messages.dequeue() == 'Brake pedal depressed'

# solution


1

### 001.004 Refactoring with mixins

`len` and `__iter__` are common to all 3 classes so far, and can be espresses as mixins. Assume `_elements` is common to all

1. Extract `__len__` and `__iter__` from earlier Queue class
1. Extend all classes created so far (Queue, Stack, PriorityQueue) with it



In [32]:
1
class IterableMixin:
    def __len__(self):
        return len(self._elements)

    def __iter__(self):
        while len(self) > 0:
            yield self.dequeue()

class Queue(IterableMixin):
    def __init__(self, *elements):
        self._elements = deque(elements)

    def enqueue(self, element):
        self._elements.append(element)

    def dequeue(self):
        return self._elements.popleft()

class Stack(Queue):
    def dequeue(self):
        return self._elements.pop()

class PriorityQueue(IterableMixin):
    def __init__(self):
        self._elements = []
        self._count = count()

    def enqueue_with_priority(self, priority, value):
        """1.2.1"""
        element = (-priority, next(self._count), value)
        heappush(self._elements, element)

    def dequeue(self):
        return heappop(self._elements)[-1]

fifo = Queue(1, 2, 3)
assert len(fifo) == 3
assert [1,2,3] == [el for el in fifo]
assert len(fifo) == 0

lifo = Stack(1, 2, 3)
assert len(lifo) == 3
assert [3,2,1] == [el for el in lifo]
assert len(fifo) == 0

messages = PriorityQueue()
messages.enqueue_with_priority(IMPORTANT, "Windshield wipers turned on")
messages.enqueue_with_priority(NEUTRAL, "Radio station tuned in")
messages.enqueue_with_priority(CRITICAL, "Brake pedal depressed")
messages.enqueue_with_priority(IMPORTANT, "Hazard lights turned on")

assert messages.dequeue() == 'Brake pedal depressed'

# solution


1